<a href="https://colab.research.google.com/github/teticio/Deej-A.I./blob/master/Spotify_Discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to discover new music on Spotify with Artificial Inteligence
## Find similar sounding music by artists you may never have heard of

In [1]:
#@title <- Press the "play" button to start!
!pip install -q spotipy
import spotipy
import spotipy.util as util
import os
import requests
import numpy as np
import pickle
import random
from IPython.core.display import display, HTML

#-----------------------------------------------------------------------------#

lookback = 3       # Number of previous tracks to consider (default 3)
size = 20          #@param {type:"integer"}
creativity = 0.5   #@param {type:"slider", min:0, max:1, step:0.01}
noise = 0          #@param {type:"slider", min:0, max:1, step:0.01}

#@markdown (Optional) If you want to be able to load your playlists into Spotify, you will need to get credentials from [here](https://developer.spotify.com/dashboard/applications)
scope = 'playlist-modify-public'
username = ''      #@param {type: "string"}
playlist_name = '' #@param {type: "string"}
#@markdown (Playlist must already exist. Uncheck the box if you want to extend it.)
replace = True     #@param {type:"boolean"}
client_id =''      #@param {type: "string"}
client_secret =''  #@param {type: "string"}
redirect_uri =''   #@param {type: "string"}

#-----------------------------------------------------------------------------#

def download_file_from_google_drive(id, destination):
    if os.path.isfile(destination):
        return None
    print(f'Downloading {destination}')
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
def most_similar(mp3tovecs, weights, positive=[], negative=[], topn=5, noise=0):
    if isinstance(positive, str):
        positive = [positive] # broadcast to list
    if isinstance(negative, str):
        negative = [negative] # broadcast to list
    similar = [[track, [0] * len(weights)] for track in mp3tovecs[0]]
    for k, mp3tovec in enumerate(mp3tovecs):
        mp3_vec_i = np.sum([mp3tovec[i] for i in positive] +
                           [-mp3tovec[i] for i in negative], axis=0)
        mp3_vec_i += np.random.normal(0, noise, len(mp3_vec_i))
        for j, track_j in enumerate(mp3tovec):
            if track_j in positive or track_j in negative:
                continue
            mp3_vec_j = mp3tovec[track_j]
            similar[j][1][k] = np.dot(mp3_vec_i, mp3_vec_j) \
                / np.linalg.norm(mp3_vec_i) / np.linalg.norm(mp3_vec_j)
    return sorted(similar, key=lambda x:-np.dot(x[1], weights))[:topn]

def most_similar_by_vec(mp3tovecs, weights, positives=None, negatives=None, topn=5, noise=0):
    similar = [[track, [0] * len(weights)] for track in mp3tovecs[0]]
    positive = negative = []
    for k, mp3tovec in enumerate(mp3tovecs):
        if positives is not None:
            positive = positives[k]
        if negatives is not None:
            negative = negatives[k]
        if isinstance(positive, str):
            positive = [positive] # broadcast to list
        if isinstance(negative, str):
            negative = [negative] # broadcast to list
        mp3_vec_i = np.sum([i for i in positive] + [-i for i in negative], axis=0)
        mp3_vec_i += np.random.normal(0, noise, len(mp3_vec_i))
        for j, track_j in enumerate(mp3tovec):
            mp3_vec_j = mp3tovec[track_j]
            similar[j][1][k] = np.dot(mp3_vec_i, mp3_vec_j) \
                / np.linalg.norm(mp3_vec_i) / np.linalg.norm(mp3_vec_j)
    return sorted(similar, key=lambda x:-np.dot(x[1], weights))[:topn]

def add_track_to_playlist(sp, username, playlist_id, track_id, replace=False):
    if sp is not None and username is not None and playlist_id is not None:
        try:
            if replace:
                sp.user_playlist_replace_tracks(username, playlist_id, [track_id])
            else:
                sp.user_playlist_add_tracks(username, playlist_id, [track_id])
        except spotipy.client.SpotifyException:
            # token has probably gone stale
            token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
            sp = spotipy.Spotify(token)
            if replace:
                sp.user_playlist_replace_tracks(username, playlist_id, [track_id])
            else:
                sp.user_playlist_add_tracks(username, playlist_id, [track_id])

def make_playlist(sp, username, playlist_id, mp3tovecs, weights, seed_tracks, \
                  size=10, lookback=3, noise=0, replace=True):
    max_tries = 100
    playlist = seed_tracks
    for i in range(0, len(seed_tracks)):
        add_track_to_playlist(sp, username, playlist_id, playlist[i], replace and len(playlist) == 1)
        display(HTML(f'<iframe src="https://open.spotify.com/embed/track/{playlist[i]}" \
                     width="100%" height="80" frameborder="0" allowtransparency="true" \
                     allow="encrypted-media"></iframe>'))
    for i in range(len(seed_tracks), size):
        similar = most_similar(mp3tovecs, weights, positive=playlist[-lookback:],
                               topn=max_tries,noise=noise)
        candidates = [candidate for candidate in similar if candidate[0] != playlist[-1]]
        for candidate in candidates:
            if not candidate[0] in playlist:
                break
        playlist.append(candidate[0])
#        print(candidate[1])
        add_track_to_playlist(sp, username, playlist_id, playlist[-1])
        display(HTML(f'<iframe src="https://open.spotify.com/embed/track/{playlist[-1]}" \
                     width="100%" height="80" frameborder="0" allowtransparency="true" \
                     allow="encrypted-media"></iframe>'))
    return playlist

# create a musical journey between given track "waypoints"
def join_the_dots(sp, username, playlist_id, mp3tovecs, weights, ids, \
                  n=5, noise=0, replace=True):
    max_tries = 100
    playlist = []
    end = start = ids[0]
    start_vec = [mp3tovec[start] for k, mp3tovec in enumerate(mp3tovecs)]
    for end in ids[1:]:
        end_vec = [mp3tovec[end] for k, mp3tovec in enumerate(mp3tovecs)]
        playlist.append(start)
        add_track_to_playlist(sp, username, playlist_id, playlist[-1], replace and len(playlist) == 1)
        display(HTML(f'<iframe src="https://open.spotify.com/embed/track/{playlist[-1]}" \
                     width="100%" height="80" frameborder="0" allowtransparency="true" \
                     allow="encrypted-media"></iframe>'))
        for i in range(n):
            similar = most_similar_by_vec(mp3tovecs, weights,
                                          [[(n-i+1)/n * start_vec[k] +
                                            (i+1)/n * end_vec[k]]
                                           for k in range(len(mp3tovecs))],
                                          topn=max_tries, noise=noise)
            candidates = [candidate for candidate in similar
                          if candidate[0] != playlist[-1]]
            for candidate in candidates:
                if not candidate[0] in playlist and candidate[0] != end:
                    break
            playlist.append(candidate[0])
#            print(candidate[1])
            add_track_to_playlist(sp, username, playlist_id, playlist[-1])
            display(HTML(f'<iframe src="https://open.spotify.com/embed/track/{playlist[-1]}" \
                         width="100%" height="80" frameborder="0" allowtransparency="true" \
                         allow="encrypted-media"></iframe>'))
        start = end
        start_vec = end_vec
    playlist.append(end)
    add_track_to_playlist(sp, username, playlist_id, playlist[-1])
    display(HTML(f'<iframe src="https://open.spotify.com/embed/track/{playlist[-1]}" \
                 width="100%" height="80" frameborder="0" allowtransparency="true" \
                 allow="encrypted-media"></iframe>'))
    return playlist

def spotify():
    sp = playlist_id = None
    if username !='' and playlist_name != '':
        if client_id == '':
            display(HTML('Get your Spotify Credentials <a href="https://developer.spotify.com\
                         /dashboard/applications">here</a> and fill in the form...'))
            display(HTML('...or simply leave the username empty'))
            return None
        token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
        if token is not None:
            sp = spotipy.Spotify(token)
            if sp is not None:
                playlists = sp.user_playlists(username)
                if playlists is not None:
                    playlist_ids = [playlist['id'] for playlist in playlists['items'] \
                                    if playlist['name'] == playlist_name]
                    if len(playlist_ids) > 0:
                        playlist_id = playlist_ids[0]
        if playlist_id is None:
            print(f'Unable to access playlist {playlist_name} for user {username}')
        else:
            print(f'Creating playlist {playlist_name} for user {username}')
    mp3tovecs = pickle.load(open('spotifytovec.p', 'rb'))
    tracktovecs = pickle.load(open('tracktovec.p', 'rb'))
    tracks = pickle.load(open('spotify_tracks.p', 'rb'))
    print(f'Loaded data for {len(mp3tovecs)} Spotify tracks')
    user_input = input('Search keywords: ')
    input_tracks = []
    while True:
        if user_input == '':
            break
        ids = sorted([track for track in mp3tovecs
                      if all(word in tracks[track].lower()
                             for word in user_input.lower().split())],
                     key = lambda x: tracks[x])
        for i, id in enumerate(ids):
            print(f'{i+1}. {tracks[id]}')
        while True:
            user_input = input('Input track number, ENTER to finish, or search keywords: ')
            if user_input == '':
                break
            if user_input.isdigit() and len(ids) > 0:
                if int(user_input)-1 >= len(ids):
                    continue
                id = ids[int(user_input)-1]
                input_tracks.append(id)
                print(f'Added {tracks[id]} to playlist')
            else:
                break
    if len(input_tracks) == 0:
        ids = [track for track in tracks]
        input_tracks.append(ids[random.randint(0, len(tracks))])
    if len(input_tracks) > 1:
        display(HTML('<h2>Joining the dots</h2>'))
        playlist = join_the_dots(sp, username, playlist_id, [mp3tovecs, tracktovecs], \
                                 [creativity, 1-creativity], input_tracks, n=size, \
                                 noise=noise, replace=replace)
    else:
        display(HTML('<h2>Playlist</h2>'))
        playlist = make_playlist(sp, username, playlist_id, [mp3tovecs, tracktovecs], \
                                 [creativity, 1-creativity], input_tracks, size=size, \
                                 lookback=lookback, noise=noise, replace=replace)
    return playlist
       
if __name__ == "__main__":
    download_file_from_google_drive('1Mg924qqF3iDgVW5w34m6Zaki5fNBdfSy', 'spotifytovec.p')
    download_file_from_google_drive('1geEALPQTRBNUvkpI08B-oN4vsIiDTb5I', 'tracktovec.p')
    download_file_from_google_drive('1Qre4Lkym1n5UTpAveNl5ffxlaAmH1ntS', 'spotify_tracks.p')
    track_details = spotify()

Loaded data for 320949 Spotify tracks
Search keywords: mulatu tezeta
1. Mulatu Astatke - Tezeta
2. Mulatu Astatke - Tezeta (Nostalgia)
3. Mulatu Astatke - Yègellé Tezeta
4. Mulatu Astatke - Yègellé Tezeta (My Own Memory)
Input track number, ENTER to finish, or search keywords: 3
Added Mulatu Astatke - Yègellé Tezeta to playlist
Input track number, ENTER to finish, or search keywords: azymuth manh
1. Azymuth - Manhã
Input track number, ENTER to finish, or search keywords: 1
Added Azymuth - Manhã to playlist
Input track number, ENTER to finish, or search keywords: 


### Some tips...
If you choose more than one track, it will create a playlist which "joins the dots" between the tracks you select by smoothly moving from one to another. This helps give the playlist direction. If you only specify one track, it will generate each new track in the playlist based on the previous 3. You can control this number with the *lookback* parameter. The playlists are intended to be played in the order in which they are created.

The *creativity* parameter controls the mix of considering songs that tend to appear close to each other in a playlist and songs that sound similar. A value of 0 will tend to pick out similar artists, while a value of 1 will likely choose songs with a similar mood, energy, rhythm or instrumentation. A good setting is usually around 0.5. For very popular artists, you may find you have to jack up the creativity parameter if you want to get off the beaten track. On the other hand, for less distinctive types of music, you may need to reduce the creativity parameter.

Finally, you can add a sprinkling of randomness to the mix by tweaking the *noise* parameter.

### What about creating automatic playlists with my own music collection?
You can and more! Check out the complete project [here](https://github.com/teticio/Deej-A.I.).